In [6]:
import pandas as pd

df_all = pd.read_csv('./Contributions-18-02-13-clean.csv')

In [63]:
IDS_OF_INTEREST = [
    (20175032103, "Michael Dougherty"),
    (20175032321, "Amy Padden"),
    (20175032143, "Brad Levin"),
    (20175031960, "Joe Salazar")
]

In [7]:
df_all['person'] = df_all['FirstName'] + ' ' + df_all['LastName'] + ' ' + df_all['Address1'] + ' ' + df_all['City']
df_all['colorado'] = df_all.State == 'CO'

In [71]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [74]:
from IPython.display import display

def do_the_thang(df):
    contribs_by_person = df.groupby('person').ContributionAmount.sum()
    occupation_count = df.Occupation.count()
    df['Occupation'] = df['Occupation'].fillna('Undisclosed')

    print('''There are {} donations ({} unique people). {} of them have an occupation listed, {} have additional occupation comments.

The average contribution is ${:.2f}

Biggest donations:
{}

Breakdown by occupation:
{}

Number of donors at max donation ($1150): {}

Occupational breakdown for top donors (those who donated $1150):
{}

Proportion of donations (by count) from out of state: {:.2f}
Proportion of donations (by money) from out of state: {:.2f}
    '''.format(
            df.shape[0],
            df.person.nunique(),
            occupation_count,
            df.OccupationComments.count(),
            df.ContributionAmount.mean(),
            df.sort_values(by='ContributionAmount', ascending=False).head().reset_index(drop=True)[['ContributionAmount', 'FirstName', 'LastName', 'City', 'State', 'Occupation']],
            df.Occupation.value_counts(),
            (contribs_by_person == 1150).sum(),
            df[df.person.isin(contribs_by_person[contribs_by_person == 1150].index)].groupby('person').first().Occupation.value_counts(),
            (df.shape[0] - df.colorado.sum()) / df.shape[0],
            df[~df.colorado].ContributionAmount.sum() / df.ContributionAmount.sum()
         ))

    asdf = df.groupby(['Occupation', 'colorado']).CO_ID.count().unstack().reset_index()
    asdf['proportion_out_of_state'] = asdf[False].fillna(0) / (asdf[False].fillna(0) + asdf[True])
    asdf['n'] = asdf[False].fillna(0) + asdf[True]
    asdf.drop([False, True], axis=1, inplace=True)

    print("\nProportion of donors from out of state, by occupation:")
    display(asdf.sort_values(by='proportion_out_of_state', ascending=False).reset_index(drop=True))
    
    print("\nSummary statistics for different occupations:")
    display(df.groupby("Occupation").ContributionAmount.agg({'mean': 'mean', 'n': 'count', 'sum': 'sum'}).sort_values(by='sum', ascending=False))

In [75]:
for coid, person in IDS_OF_INTEREST:
    print("\n\n\nSUMMARY FOR {} (Secretary of State ID: {})".format(person, coid))
    do_the_thang(df[df.CO_ID == coid])




SUMMARY FOR Michael Dougherty (Secretary of State ID: 20175032103)
There are 941 donations (625 unique people). 931 of them have an occupation listed, 15 have additional occupation comments.

The average contribution is $147.41

Biggest donations:
   ContributionAmount FirstName                   LastName      City State  \
0             15000.0       NaN  COLORADO DEMOCRATIC PARTY    DENVER    CO   
1              1150.0   MELISSA                  BEARDMORE   CROFTON    MD   
2              1150.0  DANIELLE                  DOUGHERTY   BOULDER    CO   
3              1150.0     KARYN                   KORNFELD  NEW YORK    NY   
4              1150.0    STEVEN                      KOBRE  NEW YORK    NY   

               Occupation  
0             Undisclosed  
1       Teacher/Education  
2      Healthcare/Medical  
3  Not Currently Employed  
4          Attorney/Legal  

Breakdown by occupation:
Attorney/Legal                           441
Not Currently Employed                   

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


colorado,Occupation,proportion_out_of_state,n
0,Office and Administrative Occupations,0.800000,5.0
1,Real Estate Professional,0.680000,25.0
2,Other,0.600000,15.0
3,Financial/Investment,0.578947,19.0
4,Media/Public Relations,0.555556,9.0
5,Teacher/Education,0.510204,49.0
6,Attorney/Legal,0.503401,441.0
7,Construction/Engineering,0.500000,8.0
8,General Business,0.500000,36.0
9,Not Currently Employed,0.446043,139.0



Summary statistics for different occupations:


,sum,mean,n
Occupation,,,
Attorney/Legal,57777.31,131.014308,441
Not Currently Employed,22030.00,158.489209,139
Undisclosed,16569.00,1656.900000,10
Teacher/Education,7060.00,144.081633,49
Healthcare/Medical,6492.12,231.861429,28
General Business,5144.29,142.896944,36
Consultant,4830.00,112.325581,43
Real Estate Professional,4150.00,166.000000,25
Financial/Investment,3850.00,202.631579,19


...

,sum,mean,n
Occupation,,,
Attorney/Legal,99375.00,312.500000,318
Undisclosed,60487.97,1890.249063,32
Retired,29750.00,457.692308,65
General Business,11900.00,425.000000,28
Real Estate Professional,11600.00,682.352941,17
Not Currently Employed,10150.00,563.888889,18
Consultant,9377.00,275.794118,34
Financial/Investment,6880.00,382.222222,18
Healthcare/Medical,5500.00,366.666667,15





SUMMARY FOR Joe Salazar (Secretary of State ID: 20175031960)
There are 1033 donations (782 unique people). 347 of them have an occupation listed, 2 have additional occupation comments.

The average contribution is $56.28

Biggest donations:
   ContributionAmount    FirstName                 LastName         City  \
0             3418.07          NaN  FRIENDS FOR JOE SALAZAR     THORNTON   
1             1150.00         KYLE                 ZEPPELIN       DENVER   
2             1150.00        ANDRA                 ZEPPELIN       DENVER   
3             1150.00  LORENZOALAN                 TRUJILLO  WESTMINSTER   
4             1100.00      NOOSHIN                  FARJADI  WESTMINSTER   

  State                Occupation  
0    CO               Undisclosed  
1    CO  Construction/Engineering  
2    CO    Media/Public Relations  
3    CO            Attorney/Legal  
4    CO          General Business  

Breakdown by occupation:
Undisclosed                              686
Not Currentl

colorado,Occupation,proportion_out_of_state,n
0,Undisclosed,0.294461,686.0
1,Retail Sales,0.250000,8.0
2,Science/Technology,0.250000,4.0
3,Attorney/Legal,0.095238,21.0
4,Construction/Engineering,0.055556,18.0
5,Consultant,0.052632,19.0
6,Not Currently Employed,0.044944,89.0
7,Teacher/Education,0.037037,27.0
8,Food Services,0.000000,4.0
9,Retired,0.000000,42.0



Summary statistics for different occupations:


,sum,mean,n
Occupation,,,
Undisclosed,20331.67,29.638003,686
Not Currently Employed,7895.00,88.707865,89
General Business,5215.00,162.968750,32
Retired,4278.80,101.876190,42
Attorney/Legal,3362.00,160.095238,21
Government/Civil,3202.00,100.062500,32
Teacher/Education,2817.00,104.333333,27
Construction/Engineering,2359.00,131.055556,18
Consultant,2038.00,107.263158,19
